In [1]:
#Import Dependencies
import pandas as pd
import sqlite3
from sqlite3 import Error

In [3]:
#Connection to sqlite db
con = sqlite3.connect('../../DataBase/sqlite_db.sqlite')
cursor = con.cursor()

In [4]:
# Query db w/ joins to get full stat tables
positions = ['qb', 'rb', 'te', 'wr']
df_list = []
for x in positions:
    df = f'{x}_df'
    table = f'{x.upper()}_Stats'
    table2 = f'{x.upper()}_Extra_Stats'
    df = pd.DataFrame(pd.read_sql_query(f'select * from {table} as a inner join {table2} as b on a.PlayerID = b.PlayerPlayerId and a.Year=b.Year', con))
    df_list.append(df)

In [5]:
len(df_list)

4

In [6]:
type(df_list[1])

pandas.core.frame.DataFrame

In [7]:
#Create empty dbs
qb_df = pd.DataFrame()
rb_df = pd.DataFrame()
te_df = pd.DataFrame()
wr_df = pd.DataFrame()

In [8]:
qb_df = pd.concat([df_list[0], qb_df], ignore_index=True)
rb_df = pd.concat([df_list[1], rb_df], ignore_index=True)
te_df = pd.concat([df_list[2], te_df], ignore_index=True)
wr_df = pd.concat([df_list[3], wr_df], ignore_index=True)


In [9]:
qb_df.head()

,index,Rank,PlayerID,Name,Team,Position,Played,PassingCompletions,PassingAttempts,PassingCompletionPercentage,...,MoneyThrows,TouchdownRate,ProtectionRate,PressuredCompletionPercentage,AirYards,AirYardsPerGame,AirYardsPerAttempt,DeepBallAttempts,DeepBallCompletionPercentage,Year
0,0,1,19801,Josh Allen,BUF,QB,17,409,646,63.3,...,29,5.58,77.94,37.83,2785.0,163.82,4.31,80,42.50,2021
1,1,2,21681,Justin Herbert,LAC,QB,17,443,672,65.9,...,30,5.65,88.62,41.34,2678.0,157.52,3.98,71,43.66,2021
2,2,3,4314,Tom Brady,TB,QB,17,485,719,67.5,...,44,5.98,92.98,46.84,2792.0,164.23,3.88,87,40.22,2021
3,3,4,18890,Patrick Mahomes,KC,QB,17,436,658,66.3,...,32,5.62,85.27,40.65,2219.0,130.52,3.37,79,36.70,2021
4,4,5,2593,Aaron Rodgers,GB,QB,16,366,531,68.9,...,37,6.96,88.94,39.75,1956.0,122.25,3.68,69,39.13,2021


In [10]:
qb_df.columns

Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'PassingCompletions', 'PassingAttempts', 'PassingCompletionPercentage',
       'PassingYards', 'PassingYardsPerAttempt', 'PassingTouchdowns',
       'PassingInterceptions', 'PassingRating', 'RushingAttempts',
       'RushingYards', 'RushingYardsPerAttempt', 'RushingTouchdowns',
       'FantasyPointsPerGame', 'FantasyPoints', 'Year', 'index', 'Rank',
       'PlayerPlayerId', 'PlayerShortName', 'PlayerAgeExact',
       'PassAttemptsPerGame', 'PassingYards', 'CompletionPercentage',
       'PassingTouchdowns', 'Interceptions', 'MoneyThrows', 'TouchdownRate',
       'ProtectionRate', 'PressuredCompletionPercentage', 'AirYards',
       'AirYardsPerGame', 'AirYardsPerAttempt', 'DeepBallAttempts',
       'DeepBallCompletionPercentage', 'Year'],
      dtype='object')

In [11]:
''' 
Duplicate columns creted  using this reference:
https://thispointer.com/how-to-find-drop-duplicate-columns-in-a-dataframe-python-pandas/
'''
def getDuplicateColumns(df):
  
    # Create an empty set
    duplicateColumnNames = set()
      
    # Iterate through all the columns 
    # of dataframe
    for x in range(df.shape[1]):
          
        # Take column at xth index.
        col = df.iloc[:, x]
          
        # Iterate through all the columns in
        # DataFrame from (x + 1)th index to
        # last index
        for y in range(x + 1, df.shape[1]):
              
            # Take column at yth index.
            otherCol = df.iloc[:, y]
               # Check if two columns at x & y
            # index are equal or not,
            # if equal then adding 
            # to the set
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
                  
    # Return list of unique column names 
    # whose contents are duplicates.
    return list(duplicateColumnNames)

In [12]:
dupColumnsNames = []
for x in range(0, len(positions)):
    col_name = f'dupColumns_{positions[x]}'
    dupColumnsNames.append(col_name)


In [13]:
dupColumns = []
for i in range(0, len(dupColumnsNames)):
    dupColumns.append(getDuplicateColumns(df_list[i]))
dupColumns

[['Interceptions', 'PassingTouchdowns', 'PlayerPlayerId', 'Year'],
 ['Receptions', 'PlayerPlayerId', 'Year'],
 ['PlayerPlayerId', 'Receptions', 'Targets.1', 'Year', 'ReceivingTDs'],
 ['Receptions', 'Targets.1', 'PlayerPlayerId', 'Year']]

In [14]:
for frame in df_list:
    print(frame.columns)


Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'PassingCompletions', 'PassingAttempts', 'PassingCompletionPercentage',
       'PassingYards', 'PassingYardsPerAttempt', 'PassingTouchdowns',
       'PassingInterceptions', 'PassingRating', 'RushingAttempts',
       'RushingYards', 'RushingYardsPerAttempt', 'RushingTouchdowns',
       'FantasyPointsPerGame', 'FantasyPoints', 'Year', 'index', 'Rank',
       'PlayerPlayerId', 'PlayerShortName', 'PlayerAgeExact',
       'PassAttemptsPerGame', 'PassingYards', 'CompletionPercentage',
       'PassingTouchdowns', 'Interceptions', 'MoneyThrows', 'TouchdownRate',
       'ProtectionRate', 'PressuredCompletionPercentage', 'AirYards',
       'AirYardsPerGame', 'AirYardsPerAttempt', 'DeepBallAttempts',
       'DeepBallCompletionPercentage', 'Year'],
      dtype='object')
Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       '

In [15]:
for frame in df_list:
    print(len(frame.columns))

42
44
45
45


In [16]:
df_list2 = []
for frame in df_list:
    frame2 = frame.loc[:,~frame.columns.duplicated()].copy()
    df_list2.append(frame2)

In [17]:
for frame in df_list2:
    print(len(frame.columns))

37
37
40
40


In [18]:
for frame in df_list2:
    print(frame.columns)

Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'PassingCompletions', 'PassingAttempts', 'PassingCompletionPercentage',
       'PassingYards', 'PassingYardsPerAttempt', 'PassingTouchdowns',
       'PassingInterceptions', 'PassingRating', 'RushingAttempts',
       'RushingYards', 'RushingYardsPerAttempt', 'RushingTouchdowns',
       'FantasyPointsPerGame', 'FantasyPoints', 'Year', 'PlayerPlayerId',
       'PlayerShortName', 'PlayerAgeExact', 'PassAttemptsPerGame',
       'CompletionPercentage', 'Interceptions', 'MoneyThrows', 'TouchdownRate',
       'ProtectionRate', 'PressuredCompletionPercentage', 'AirYards',
       'AirYardsPerGame', 'AirYardsPerAttempt', 'DeepBallAttempts',
       'DeepBallCompletionPercentage'],
      dtype='object')
Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       'RushingTouchdowns', 'ReceivingTargets', 'Receptions', 'ReceivingYards

In [19]:
dupColumns2 = []
for i in range(0, len(dupColumnsNames)):
    dupColumns2.append(getDuplicateColumns(df_list2[i]))
dupColumns2

[['Interceptions', 'PlayerPlayerId'],
 ['PlayerPlayerId'],
 ['Targets.1', 'ReceivingTDs', 'PlayerPlayerId'],
 ['Targets.1', 'PlayerPlayerId']]

In [20]:
for i in range(0, len(df_list2)):
    df_list2[i].drop(columns = dupColumns2[i], axis = 1, inplace = True)

In [21]:
for frame in df_list2:
    print(frame.columns)

Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'PassingCompletions', 'PassingAttempts', 'PassingCompletionPercentage',
       'PassingYards', 'PassingYardsPerAttempt', 'PassingTouchdowns',
       'PassingInterceptions', 'PassingRating', 'RushingAttempts',
       'RushingYards', 'RushingYardsPerAttempt', 'RushingTouchdowns',
       'FantasyPointsPerGame', 'FantasyPoints', 'Year', 'PlayerShortName',
       'PlayerAgeExact', 'PassAttemptsPerGame', 'CompletionPercentage',
       'MoneyThrows', 'TouchdownRate', 'ProtectionRate',
       'PressuredCompletionPercentage', 'AirYards', 'AirYardsPerGame',
       'AirYardsPerAttempt', 'DeepBallAttempts',
       'DeepBallCompletionPercentage'],
      dtype='object')
Index(['index', 'Rank', 'PlayerID', 'Name', 'Team', 'Position', 'Played',
       'RushingAttempts', 'RushingYards', 'RushingYardsPerAttempt',
       'RushingTouchdowns', 'ReceivingTargets', 'Receptions', 'ReceivingYards',
       'ReceivingTouchdowns', 'F

In [22]:
for frame in df_list2:
    frame.drop(columns = 'index', axis = 1, inplace= True)

In [23]:
for frame in df_list2:
    print(frame.isnull().sum())

Rank                             0
PlayerID                         0
Name                             0
Team                             0
Position                         0
Played                           0
PassingCompletions               0
PassingAttempts                  0
PassingCompletionPercentage      0
PassingYards                     0
PassingYardsPerAttempt           0
PassingTouchdowns                0
PassingInterceptions             0
PassingRating                    0
RushingAttempts                  0
RushingYards                     0
RushingYardsPerAttempt           0
RushingTouchdowns                0
FantasyPointsPerGame             0
FantasyPoints                    0
Year                             0
PlayerShortName                  0
PlayerAgeExact                   0
PassAttemptsPerGame              0
CompletionPercentage             0
MoneyThrows                      0
TouchdownRate                    0
ProtectionRate                   0
PressuredCompletionP

In [24]:
for frame in df_list2:
    print(frame.head(1))

   Rank  PlayerID        Name Team Position  Played  PassingCompletions  \
0     1     19801  Josh Allen  BUF       QB      17                 409   

   PassingAttempts  PassingCompletionPercentage  PassingYards  ...  \
0              646                         63.3          4407  ...   

   CompletionPercentage  MoneyThrows  TouchdownRate  ProtectionRate  \
0                 63.41           29           5.58           77.94   

   PressuredCompletionPercentage  AirYards  AirYardsPerGame  \
0                          37.83    2785.0           163.82   

   AirYardsPerAttempt  DeepBallAttempts  DeepBallCompletionPercentage  
0                4.31                80                          42.5  

[1 rows x 34 columns]
   Rank  PlayerID             Name Team Position  Played  RushingAttempts  \
0     1     21682  Jonathan Taylor  IND       RB      17              332   

   RushingYards  RushingYardsPerAttempt  RushingTouchdowns  ...  SnapShare  \
0          1811                     5.

In [25]:
qb_full_stats = df_list2[0]
rb_full_stats = df_list2[1]
te_full_stats = df_list2[2]
wr_full_stats = df_list2[3]

In [28]:
wr_full_stats.drop(columns='HogRate', axis = 1, inplace=True)
te_full_stats.drop(columns='HogRate', axis = 1, inplace=True)

In [30]:
wr_full_stats.isnull().sum()

Rank                          0
PlayerID                      0
Name                          0
Team                          0
Position                      0
Played                        0
ReceivingTargets              0
Receptions                    0
ReceptionPercentage           0
ReceivingYards                0
ReceivingTouchdowns           0
ReceivingLong                 0
ReceivingYardsPerTarget       0
ReceivingYardsPerReception    0
RushingAttempts               0
RushingYards                  0
RushingYardsPerAttempt        0
RushingTouchdowns             0
Fumbles                       0
FumblesLost                   0
FantasyPointsPerGame          0
FantasyPoints                 0
Year                          0
PlayerShortName               0
Targets                       0
ReceivingTDs                  0
SnapShare                     0
AverageTargetDistance         0
AirYards                      0
AirYardsPerGame               0
AirYardsPerReception          0
RedZoneT

In [31]:
te_full_stats.isnull().sum()

Rank                          0
PlayerID                      0
Name                          0
Team                          0
Position                      0
Played                        0
ReceivingTargets              0
Receptions                    0
ReceptionPercentage           0
ReceivingYards                0
ReceivingTouchdowns           0
ReceivingLong                 0
ReceivingYardsPerTarget       0
ReceivingYardsPerReception    0
RushingAttempts               0
RushingYards                  0
RushingYardsPerAttempt        0
RushingTouchdowns             0
Fumbles                       0
FumblesLost                   0
FantasyPointsPerGame          0
FantasyPoints                 0
Year                          0
PlayerShortName               0
Targets                       0
SnapShare                     0
AverageTargetDistance         0
AirYards                      0
AirYardsPerGame               0
AirYardsPerReception          0
RedZoneTargets                0
EndzoneT

In [32]:
rb_full_stats.isnull().sum()

Rank                         0
PlayerID                     0
Name                         0
Team                         0
Position                     0
Played                       0
RushingAttempts              0
RushingYards                 0
RushingYardsPerAttempt       0
RushingTouchdowns            0
ReceivingTargets             0
Receptions                   0
ReceivingYards               0
ReceivingTouchdowns          0
Fumbles                      0
FumblesLost                  0
FantasyPointsPerGame         0
FantasyPoints                0
Year                         0
PlayerShortName              0
PlayerAgeExact               0
Carries                      0
YardsPerCarry               30
Targets                      0
ReceivingTDs                 0
SnapShare                    0
OpportunityShare             6
ShotgunCarryRate            58
ShotgunYardsPerCarry        67
UnderCenterCarryRate        58
UnderCenterYardsPerCarry    40
AverageDefendersInTheBox    58
StackedF

In [33]:
rb_full_stats[rb_full_stats["Carries"] == 0].count()

Rank                        52
PlayerID                    52
Name                        52
Team                        52
Position                    52
Played                      52
RushingAttempts             52
RushingYards                52
RushingYardsPerAttempt      52
RushingTouchdowns           52
ReceivingTargets            52
Receptions                  52
ReceivingYards              52
ReceivingTouchdowns         52
Fumbles                     52
FumblesLost                 52
FantasyPointsPerGame        52
FantasyPoints               52
Year                        52
PlayerShortName             52
PlayerAgeExact              52
Carries                     52
YardsPerCarry               28
Targets                     52
ReceivingTDs                52
SnapShare                   52
OpportunityShare            52
ShotgunCarryRate             0
ShotgunYardsPerCarry        28
UnderCenterCarryRate         0
UnderCenterYardsPerCarry    28
AverageDefendersInTheBox     0
StackedF

In [35]:
null_columns = ["LightFrontCarryRate", "BaseFrontCarryRate", "StackedFrontCarryRate", "UnderCenterYardsPerCarry", "UnderCenterCarryRate", "ShotgunYardsPerCarry", "ShotgunCarryRate"]
for column in null_columns: 
     rb_full_stats[rb_full_stats["Carries"] == 0] = rb_full_stats[rb_full_stats["Carries"] == 0].fillna(value = 0)

In [36]:
rb_full_stats[rb_full_stats["Carries"] == 0].count()

Rank                        52
PlayerID                    52
Name                        52
Team                        52
Position                    52
Played                      52
RushingAttempts             52
RushingYards                52
RushingYardsPerAttempt      52
RushingTouchdowns           52
ReceivingTargets            52
Receptions                  52
ReceivingYards              52
ReceivingTouchdowns         52
Fumbles                     52
FumblesLost                 52
FantasyPointsPerGame        52
FantasyPoints               52
Year                        52
PlayerShortName             52
PlayerAgeExact              52
Carries                     52
YardsPerCarry               52
Targets                     52
ReceivingTDs                52
SnapShare                   52
OpportunityShare            52
ShotgunCarryRate            52
ShotgunYardsPerCarry        52
UnderCenterCarryRate        52
UnderCenterYardsPerCarry    52
AverageDefendersInTheBox    52
StackedF

In [37]:
rb_full_stats.isnull().sum()

Rank                         0
PlayerID                     0
Name                         0
Team                         0
Position                     0
Played                       0
RushingAttempts              0
RushingYards                 0
RushingYardsPerAttempt       0
RushingTouchdowns            0
ReceivingTargets             0
Receptions                   0
ReceivingYards               0
ReceivingTouchdowns          0
Fumbles                      0
FumblesLost                  0
FantasyPointsPerGame         0
FantasyPoints                0
Year                         0
PlayerShortName              0
PlayerAgeExact               0
Carries                      0
YardsPerCarry                6
Targets                      0
ReceivingTDs                 0
SnapShare                    0
OpportunityShare             6
ShotgunCarryRate             6
ShotgunYardsPerCarry        43
UnderCenterCarryRate         6
UnderCenterYardsPerCarry    16
AverageDefendersInTheBox     6
StackedF

In [42]:
pd.set_option('display.max_columns', None)

In [50]:
rb_full_stats[rb_full_stats["YardsPerCarry"].isnull()]
rb_full_stats.drop(columns = ["Carries", "YardsPerCarry", "Targets"], axis = 1, inplace = True)

In [51]:
rb_full_stats[rb_full_stats["ShotgunYardsPerCarry"].isnull()]

,Rank,PlayerID,Name,Team,Position,Played,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGame,FantasyPoints,Year,PlayerShortName,PlayerAgeExact,ReceivingTDs,SnapShare,OpportunityShare,ShotgunCarryRate,ShotgunYardsPerCarry,UnderCenterCarryRate,UnderCenterYardsPerCarry,AverageDefendersInTheBox,StackedFrontCarryRate,BaseFrontCarryRate,LightFrontCarryRate
234,73,15150,Cordarrelle Patterson,CHI,RB,16,64,232,3.6,1,25,21,132,0,0,0,2.6,42.4,2020,C. Patterson,31.3,0,18.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,100,20819,Alec Ingold,LV,FB,16,3,4,1.3,0,17,12,110,1,0,0,1.1,17.4,2020,A. Ingold,26.0,1,19.30,5.7,0.0,NaN,100.0,1.3,7.67,33.3,66.7,0.0
266,106,22310,Gabe Nabers,LAC,FB,16,2,7,3.5,0,7,5,25,2,0,0,1.0,15.2,2020,G. Nabers,24.7,2,14.50,3.3,0.0,NaN,100.0,3.5,7.50,50.0,50.0,0.0
285,128,13418,Anthony Sherman,KC,FB,13,3,8,2.7,0,2,1,5,1,0,0,0.6,7.3,2020,A. Sherman,33.6,1,11.40,4.0,0.0,NaN,100.0,2.7,8.00,66.7,33.3,0.0
292,137,16928,Michael Burton,NO,FB,15,7,18,2.6,0,4,4,28,0,0,0,0.3,4.6,2020,M. Burton,30.4,0,21.90,5.1,0.0,NaN,100.0,2.6,7.43,57.1,28.6,14.3
299,144,19072,Alex Armah,CAR,FB,16,6,9,1.5,0,9,5,18,0,0,0,0.2,2.7,2020,A. Armah,28.1,0,13.90,5.0,0.0,NaN,100.0,1.5,7.83,83.3,16.7,0.0
302,147,18093,Andy Janovich,CLE,FB,14,2,4,2.0,0,5,2,13,0,0,0,0.1,1.7,2020,A. Janovich,29.1,0,16.80,3.2,0.0,NaN,100.0,2.0,7.50,50.0,50.0,0.0
391,63,18464,J.D. McKissic,DET,RB,16,38,205,5.4,0,42,34,233,1,0,0,3.1,49.8,2019,J. McKissic,28.9,1,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,91,18652,C.J. Ham,MIN,FB,16,7,17,2.4,0,26,17,149,1,1,1,1.3,20.6,2019,C. Ham,29.0,1,22.50,7.6,0.0,NaN,114.3,2.0,8.29,42.9,42.9,28.6
421,93,15150,Cordarrelle Patterson,CHI,RB,16,17,103,6.1,0,17,11,83,0,0,0,1.2,18.6,2019,C. Patterson,31.3,0,19.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
rb_full_stats.isnull().sum()

Rank                         0
PlayerID                     0
Name                         0
Team                         0
Position                     0
Played                       0
RushingAttempts              0
RushingYards                 0
RushingYardsPerAttempt       0
RushingTouchdowns            0
ReceivingTargets             0
Receptions                   0
ReceivingYards               0
ReceivingTouchdowns          0
Fumbles                      0
FumblesLost                  0
FantasyPointsPerGame         0
FantasyPoints                0
Year                         0
PlayerShortName              0
PlayerAgeExact               0
ReceivingTDs                 0
SnapShare                    0
OpportunityShare             6
ShotgunCarryRate             6
ShotgunYardsPerCarry        43
UnderCenterCarryRate         6
UnderCenterYardsPerCarry    16
AverageDefendersInTheBox     6
StackedFrontCarryRate        6
BaseFrontCarryRate           6
LightFrontCarryRate          6
dtype: i

In [55]:
rb_full_stats[rb_full_stats["ShotgunYardsPerCarry"].isnull()]


,Rank,PlayerID,Name,Team,Position,Played,RushingAttempts,RushingYards,RushingYardsPerAttempt,RushingTouchdowns,ReceivingTargets,Receptions,ReceivingYards,ReceivingTouchdowns,Fumbles,FumblesLost,FantasyPointsPerGame,FantasyPoints,Year,PlayerShortName,PlayerAgeExact,ReceivingTDs,SnapShare,OpportunityShare,ShotgunCarryRate,ShotgunYardsPerCarry,UnderCenterCarryRate,UnderCenterYardsPerCarry,AverageDefendersInTheBox,StackedFrontCarryRate,BaseFrontCarryRate,LightFrontCarryRate
234,73,15150,Cordarrelle Patterson,CHI,RB,16,64,232,3.6,1,25,21,132,0,0,0,2.6,42.4,2020,C. Patterson,31.3,0,18.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,100,20819,Alec Ingold,LV,FB,16,3,4,1.3,0,17,12,110,1,0,0,1.1,17.4,2020,A. Ingold,26.0,1,19.30,5.7,0.0,NaN,100.0,1.3,7.67,33.3,66.7,0.0
266,106,22310,Gabe Nabers,LAC,FB,16,2,7,3.5,0,7,5,25,2,0,0,1.0,15.2,2020,G. Nabers,24.7,2,14.50,3.3,0.0,NaN,100.0,3.5,7.50,50.0,50.0,0.0
285,128,13418,Anthony Sherman,KC,FB,13,3,8,2.7,0,2,1,5,1,0,0,0.6,7.3,2020,A. Sherman,33.6,1,11.40,4.0,0.0,NaN,100.0,2.7,8.00,66.7,33.3,0.0
292,137,16928,Michael Burton,NO,FB,15,7,18,2.6,0,4,4,28,0,0,0,0.3,4.6,2020,M. Burton,30.4,0,21.90,5.1,0.0,NaN,100.0,2.6,7.43,57.1,28.6,14.3
299,144,19072,Alex Armah,CAR,FB,16,6,9,1.5,0,9,5,18,0,0,0,0.2,2.7,2020,A. Armah,28.1,0,13.90,5.0,0.0,NaN,100.0,1.5,7.83,83.3,16.7,0.0
302,147,18093,Andy Janovich,CLE,FB,14,2,4,2.0,0,5,2,13,0,0,0,0.1,1.7,2020,A. Janovich,29.1,0,16.80,3.2,0.0,NaN,100.0,2.0,7.50,50.0,50.0,0.0
391,63,18464,J.D. McKissic,DET,RB,16,38,205,5.4,0,42,34,233,1,0,0,3.1,49.8,2019,J. McKissic,28.9,1,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
419,91,18652,C.J. Ham,MIN,FB,16,7,17,2.4,0,26,17,149,1,1,1,1.3,20.6,2019,C. Ham,29.0,1,22.50,7.6,0.0,NaN,114.3,2.0,8.29,42.9,42.9,28.6
421,93,15150,Cordarrelle Patterson,CHI,RB,16,17,103,6.1,0,17,11,83,0,0,0,1.2,18.6,2019,C. Patterson,31.3,0,19.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
qb_full_stats.to_sql("QB_Full_Stats", con, if_exists="replace")
wr_full_stats.to_sql("WR_Full_Stats", con, if_exists="replace")
rb_full_stats.to_sql("RB_Full_Stats", con, if_exists="replace")
te_full_stats.to_sql("TE_Full_Stats", con, if_exists="replace")


In [57]:
# Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""

In [59]:
# executing our sql query
cursor.execute(sql_query)
print("List of tables\n")
    
# printing all tables list
print(cursor.fetchall())

List of tables

[('QuickStats',), ('Teams',), ('QB_Stats',), ('QB_Extra_Stats',), ('WR_Stats',), ('WR_Extra_Stats',), ('RB_Stats',), ('RB_Extra_Stats',), ('TE_Stats',), ('TE_Extra_Stats',), ('QB_Full_Stats',), ('WR_Full_Stats',), ('RB_Full_Stats',), ('TE_Full_Stats',)]
